In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf

import pip
pip.main(['install', 'xgboost'])
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import matplotlib.pyplot as plt

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
bat_charge_path = r"/home/yamukelani/source_code/msc_research/rul_prediction/src/datasets/all_batteries_charging_data.csv"
bat_discharge_path = r"/home/yamukelani/source_code/msc_research/rul_prediction/src/datasets/all_batteries_discharging_data.csv"

battery_charging = pd.read_csv(bat_charge_path)
battery_discharging = pd.read_csv(bat_discharge_path)

In [3]:
bat_discharge_sel = battery_discharging[['Ambient_temperature', 'Voltage_measured',
       'Current_measured', 'Temperature_measured', 'Current_load',
       'Voltage_load', 'Discharge_time', 'Capacity']].set_index('Discharge_time')
bat_discharge_sel


,Ambient_temperature,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Capacity
Discharge_time,,,,,,,
0.000,24,4.191492,-0.004902,24.330034,-0.0006,0.000,1.856487
16.781,24,4.190749,-0.001478,24.325993,-0.0006,4.206,1.856487
35.703,24,3.974871,-2.012528,24.389085,-1.9982,3.062,1.856487
53.781,24,3.951717,-2.013979,24.544752,-1.9982,3.030,1.856487
71.922,24,3.934352,-2.011144,24.731385,-1.9982,3.011,1.856487
...,...,...,...,...,...,...,...
2686.359,24,3.443760,-0.002426,35.383979,0.0006,0.000,1.341051
2700.546,24,3.453271,-0.000981,35.179732,0.0006,0.000,1.341051
2714.640,24,3.461963,0.000209,34.977000,0.0006,0.000,1.341051


# Split the battery discharging data into training and test datasets. 

In [4]:
bat_discharge_train, bat_discharge_test = train_test_split(bat_discharge_sel, test_size=0.3)
X_train = bat_discharge_train[['Ambient_temperature', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Current_load', 'Voltage_load']]
y_train = bat_discharge_train[['Capacity']]
X_test = bat_discharge_test[['Ambient_temperature', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Current_load', 'Voltage_load']]
y_test = bat_discharge_test[['Capacity']]

In [5]:
bat_discharge_train.head(10)

,Ambient_temperature,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Capacity
Discharge_time,,,,,,,
1012.015,24,3.649749,-2.013449,29.392095,1.9982,2.691,1.741850
668.875,24,3.649056,-2.005839,28.030751,1.9986,2.741,1.447866
984.485,24,3.579191,-2.011532,30.971798,1.9982,2.612,1.485904
50.484,24,3.926122,-2.008494,23.161182,1.9986,2.979,1.406448
806.500,24,3.561970,-2.011794,30.891831,1.9982,2.589,1.309015
2764.531,24,2.994951,0.000951,41.067551,0.0006,0.001,1.482078
1546.781,24,3.545289,-2.010194,33.160621,1.9992,2.640,1.823719
1340.797,24,3.469346,-2.009414,30.133956,1.9988,2.559,1.426843
1035.469,24,3.562610,-2.010595,29.173368,1.9986,2.652,1.480738


In [6]:
bat_discharge_test.head(10)

,Ambient_temperature,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Capacity
Discharge_time,,,,,,,
225.563,24,3.721537,-2.009412,26.772211,1.9988,2.809,1.289805
3328.828,24,3.339674,-2.012212,36.929766,-1.9990,2.446,2.025140
291.218,24,3.807107,-1.985759,26.892396,2.0000,2.872,1.492414
2080.656,24,3.509104,-1.990522,35.020260,2.0000,2.568,1.815076
1228.000,24,3.500210,-2.012984,31.015814,1.9990,2.593,1.551171
1147.203,24,3.497554,-2.009465,30.261666,1.9988,2.590,1.393567
3450.891,24,2.252022,-1.990536,39.465246,-2.0000,1.418,1.881472
87.656,24,3.876819,-2.005500,24.024656,1.9988,2.941,1.373346
1381.234,24,3.575258,-2.008546,29.946029,1.9986,2.671,1.746220


In [7]:
len(bat_discharge_test)/len(bat_discharge_sel)

0.300003769094502

In [8]:
len(bat_discharge_train)/len(bat_discharge_sel)

0.699996230905498

In [9]:
len(bat_discharge_sel)

185721

In [10]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, r2_score

# Choose Kernel function

In [11]:
# Choose kernel function
kernel = ConstantKernel(constant_value=1.0) * RBF(length_scale=1.0) # Product of constant and RBF kernels

# Create and fit model

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_s=scaler.fit_transform(X_train)
X_test_s=scaler.fit_transform(X_test)
y_train_s = scaler.fit_transform(y_train)
y_test_s = scaler.fit_transform(y_test)

In [16]:
X_test_s

array([[0.00000000e+00, 7.92067894e-01, 8.22614241e-03, 2.21322535e-01,
        9.99700000e-01, 6.62812647e-01],
       [0.00000000e+00, 6.33567697e-01, 6.85279802e-03, 7.31708945e-01,
        2.50000000e-04, 5.77159037e-01],
       [0.00000000e+00, 8.27585820e-01, 1.98292580e-02, 2.27361429e-01,
        1.00000000e+00, 6.77678150e-01],
       ...,
       [0.00000000e+00, 6.94600573e-01, 7.24619365e-03, 4.15106948e-01,
        9.99750000e-01, 6.08069844e-01],
       [0.00000000e+00, 5.92157457e-01, 7.47509023e-03, 7.49915321e-01,
        9.99750000e-01, 5.50023596e-01],
       [0.00000000e+00, 9.02710541e-01, 6.26875053e-03, 1.02583690e-01,
        4.50000000e-04, 7.22038697e-01]])

In [18]:
y_train_s

array([[0.66706571],
       [0.33356931],
       [0.37671924],
       ...,
       [0.16969761],
       [0.26093544],
       [0.57843097]])

In [ ]:
# Create and fit model
# Create GaussianProcessRegressor object with kernel and noise variance
gpr_model = GaussianProcessRegressor(kernel=kernel) 
gpr_model.fit(X_train_s, y_train_s) # Fit model to training data

In [33]:
len(X_train)

404

In [35]:
type(X)

numpy.ndarray